# Exploración de datos de libros, autores y reseñas para una nueva plataforma de lectura
___



**Objetivo**:

Analizar una base de datos de libros, autores, editoriales y valoraciones de usuarios mediante consultas SQL, con el fin de identificar patrones de publicación, preferencias de los lectores y desempeño de autores y editoriales, que permitan generar una propuesta de valor para el desarrollo de un nuevo producto enfocado en lectores.

**Tareas a realizar**:

 - Determinar la cantidad de libros publicados a partir del año 2000 para identificar tendencias recientes en el mercado editorial.

 - Analizar la cantidad de reseñas y la calificación promedio de cada libro para evaluar su recepción entre los usuarios.

 - Identificar la editorial con mayor volumen de publicaciones relevantes (libros con más de 50 páginas).

 - Identificar al autor con mejor calificación promedio considerando solo libros con al menos 50 calificaciones.

 - Estudiar el comportamiento de los usuarios más activos mediante el análisis del promedio de reseñas escritas por quienes califican más de 50 libros.

In [1]:
#Se cargan las librerias a utilizar

import pandas as pd
from sqlalchemy import create_engine

In [2]:
#Se crea la conexión a la base de datos

db_config = { 'user': 'practicum_student', 
              'pwd': 'QnmDH8Sc2TQLvy2G3Vvh7', 
              'host': 'yp-trainers-practicum.cluster-czs0gxyx2d8w.us-east-1.rds.amazonaws.com', 
              'port': 5432, 
              'db': 'data-analyst-final-project-db'} 

In [3]:
connection_string = 'postgresql://{}:{}@{}:{}/{}'.format(
    db_config['user'], 
    db_config['pwd'], 
    db_config['host'], 
    db_config['port'], 
    db_config['db']
)

In [4]:
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

## Tarea 1. Determinar la cantidad de libros publicados a partir del año 2000 para identificar tendencias recientes en el mercado editorial

In [5]:
tarea_1 = '''
SELECT
    COUNT(*) AS cnt
FROM
    books
WHERE
    publication_date >= '2000-01-01'
'''

In [6]:
pd.io.sql.read_sql(tarea_1, con = engine)

,cnt
0,821


In [7]:
tarea_1_1 = '''
SELECT
    *
FROM
    books
WHERE
    publication_date >= '2000-01-01'
LIMIT 10
'''

In [8]:
pd.io.sql.read_sql(tarea_1_1, con = engine)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268
5,6,257,1st to Die (Women's Murder Club #1),424,2005-05-20,116
6,7,258,2nd Chance (Women's Murder Club #2),400,2005-05-20,116
7,8,260,4th of July (Women's Murder Club #4),448,2006-06-01,318
8,9,563,A Beautiful Mind,461,2002-02-04,104
9,10,445,A Bend in the Road,341,2005-04-01,116


De los **1,000** libros contenidos en la base de datos, **821** fueron publicados después del 1 de enero del año 2000.

## Tarea 2. Analizar la cantidad de reseñas y la calificación promedio de cada libro para evaluar su recepción entre los usuarios

In [9]:
tarea_2 = '''
SELECT
    b.title,
    COUNT(r.review_id) AS review_count,
    AVG(rt.rating) AS ratings_avg
FROM
    books AS b
INNER JOIN reviews AS r ON b.book_id = r.book_id
INNER JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY
    b.book_id
ORDER BY
    ratings_avg DESC
'''

In [10]:
pd.io.sql.read_sql(tarea_2, con = engine)

,title,review_count,ratings_avg
0,A Woman of Substance (Emma Harte Saga #1),4,5.00
1,Pop Goes the Weasel (Alex Cross #5),4,5.00
2,In the Hand of the Goddess (Song of the Liones...,6,5.00
3,Piercing the Darkness (Darkness #2),4,5.00
4,Act of Treason (Mitch Rapp #9),4,5.00
...,...,...,...
989,The World Is Flat: A Brief History of the Twen...,12,2.25
990,Junky,4,2.00
991,His Excellency: George Washington,4,2.00
992,Drowning Ruth,9,2.00


En este listado se muestra el número de reseñas por libro y su calificación promedio.

Es importante destacar que los libros no cuentan con el mismo número de reseñas, por lo que la calificación promedio debe interpretarse con cautela, considerando esta diferencia.

## Tarea 3. Identificar la editorial con mayor volumen de publicaciones relevantes (libros con más de 50 páginas)

In [11]:
tarea_3 = '''
SELECT
    b.publisher_id,
    publisher,
    COUNT(b.publisher_id) AS num_books
FROM
    books AS b
INNER JOIN publishers AS p ON b.publisher_id = p.publisher_id
WHERE
    num_pages >= 50
GROUP BY
    b.publisher_id,
    publisher
ORDER BY
    num_books DESC
LIMIT 10
'''

In [12]:
pd.io.sql.read_sql(tarea_3, con = engine)

,publisher_id,publisher,num_books
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,35,Bantam,19
5,33,Ballantine Books,19
6,45,Berkley,17
7,46,Berkley Books,14
8,284,St. Martin's Press,14
9,83,Delta,13


En este listado se presenta el **Top 10** de editoriales con libros de más de 50 páginas, así como el número de libros publicados por cada una.

## Tarea 4. Identificar al autor con mejor calificación promedio considerando solo libros con al menos 50 calificaciones

In [13]:
tarea_4 = '''
SELECT
    a.author,
    AVG(rt.rating) AS ratings_avg,
    COUNT(rt.rating) AS rating_count
FROM
    books AS b
INNER JOIN authors AS a ON b.author_id = a.author_id
INNER JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY
    a.author
HAVING
    COUNT(rt.rating) >= 50
ORDER BY
    ratings_avg DESC
LIMIT 10
'''

In [14]:
pd.io.sql.read_sql(tarea_4, con = engine)

,author,ratings_avg,rating_count
0,Diana Gabaldon,4.300000,50
1,J.K. Rowling/Mary GrandPré,4.288462,312
2,Agatha Christie,4.283019,53
3,Markus Zusak/Cao Xuân Việt Khương,4.264151,53
4,J.R.R. Tolkien,4.240964,166
5,Roald Dahl/Quentin Blake,4.209677,62
6,Louisa May Alcott,4.203704,54
7,Rick Riordan,4.130952,84
8,Arthur Golden,4.107143,56
9,Stephen King,4.009434,106


En este listado se presenta el **Top 10** de autores mejor calificados, considerando únicamente aquellos con al menos **50 calificaciones**, así como el promedio de las calificaciones otorgadas por los lectores.

## Tarea 5. Estudiar el comportamiento de los usuarios más activos mediante el análisis del promedio de reseñas escritas por quienes califican más de 50 libros

In [40]:
tarea_5 = '''
WITH avg_reviews AS (
SELECT
    reviews_ct.username,
    reviews_ct.count_text,
    rating_ct.count_books
FROM
    (SELECT
        username,
        COUNT(text) AS count_text
        FROM reviews
        GROUP BY username
        ORDER BY count_text DESC) AS reviews_ct
        RIGHT JOIN (
        SELECT
            username,
            COUNT(book_id) AS count_books
        FROM ratings
        GROUP BY username
        HAVING COUNT(book_id) > 50
        ORDER BY count_books DESC
        ) AS rating_ct
        ON reviews_ct.username = rating_ct.username
)
SELECT
    AVG(count_text) AS avg_count_text
FROM avg_reviews
'''

In [41]:
pd.io.sql.read_sql(tarea_5, con = engine)

,avg_count_text
0,24.333333


De entre los **160** usuarios más activos, definidos como aquellos que calificaron más de **50 libros**, se observa que los usuarios que escribieron al menos una reseña de texto generaron en promedio **24.33** reseñas por usuario.